In [5]:
import json
import gzip
import os
import glob
import pickle
import time
from IPython.display import clear_output
import pandas as pd

In [6]:
candidates_keywords = {'Joe Biden': ['Joe Biden','@JoeBiden', 'Biden', '#Biden2020', '#JoeBiden', '#TeamJoe', '#TellTheTruthJoe', '#BidenIsCorrupt', '#BidenisCorrupt'],
                        'Michael Bennet' : ['@MichaelBennet', 'Michael Bennet'],
                        'Michael Bloomberg' : ['@Mike2020', 'Michael Bloomberg', '#bloomberg2020'],
                        'Pete Buttigieg' : ['Pete Buttigieg','@PeteButtigieg', '#dropoutpete', '#pete2020', '#PeteForAmerica', '#PeteforPresident', '#TeamPete', '#winecave', '#winecavepete'],
                        'John Delaney' : ['@JohnDelaney', 'John Delaney'],
                        'Tulsi Gabbard' : ['Tulsi Gabbard', '@tulsigabbard', '#StandWithTulsi', '#tulsiwasright', '#tulsigabbard', '#Tulsi2020', '#Tulsi', '#tulsicoward'],
                        'Amy Klobuchar' :['Amy Klobuchar', '@amyklobuchar', '#AmyforAmerica', '#klobucharmy', '#DropOutAmy'],
                        'Deval Patrick' : ['Deval Patrick', '@DevalPatrick', '#Devalforall', '#DevalPatrick2020', '#devalpatrick'],
                        'Bernie Sanders' : ['Bernie Sanders', '@BernieSanders', '#Bernie', '#berniesanders', '#bernie2020', '#Fightfor15', '#ILikeBernie', '#NobodyLikesHim', '#NotMeUs', '#TrustBernie', '#ITrustBernie', '#VoteforBernie', '#WomenForBernie', '#blackwomenforbernie', '#berniebeatstrump', '#marxistbernie', '#berniebots'],
                        'Tom Steyer' : ['Tom Steyer', '@TomSteyer', '#momenTOM', '#TomforPresident'],
                        'Elizabeth Warren' : ['Elizabeth Warren', '@ewarren', '#imWithHer', '#TeamWarren', '#Warren', '#Warren2020', '#WarrenHarris', '#WarrenIsaSnake', '#WarrenCastro2020'],
                        'Andrew Yang' : ['Andrew Yang', '@AndrewYang', '#AmericaNeedsYang', '#AndrewYang', '#NHforYang', '#Yang2020', '#YangGang'],
                      }
                       
 

In [7]:
def load_progress():
    
    #loading the dataframe tracking the progress of sentiment analysis 
    pkl_file1 = open('analysis_progress.pkl', 'rb')
    analysis_progress = pickle.load(pkl_file1)
    pkl_file1.close()  

    #loading the daily sentiment dataframe
    if(os.path.exists('daily_candidate_sentiments_df.pkl')):
        pkl_file = open('daily_candidate_sentiments_df.pkl', 'rb')
        daily_candidate_sentiments_df = pickle.load(pkl_file)
        pkl_file.close()

    else:
        daily_candidate_sentiments_df = pd.DataFrame(columns = list(candidates_keywords.keys()))
        daily_candidate_sentiments_df.insert(0, 'Date','')

    #loading the daily tweets count dataframe   
    if(os.path.exists('daily_candidate_tweets_df.pkl')):
        pkl_file = open('daily_candidate_tweets_df.pkl', 'rb')
        daily_candidate_tweets_df = pickle.load(pkl_file)
        pkl_file.close()

    else:
        daily_candidate_tweets_df = pd.DataFrame(columns = list(candidates_keywords.keys()))
        daily_candidate_tweets_df.insert(0, 'Date','')
    return analysis_progress, daily_candidate_sentiments_df, daily_candidate_tweets_df



In [8]:
analysis_progress, daily_candidate_sentiments_df, daily_candidate_tweets_df = load_progress()

for date in analysis_progress:
    
    if(analysis_progress[date] == 'analyzed' and date not in daily_candidate_sentiments_df['Date'].values):
        
        #loading the tweets for that date
        tweet_pkl = open('Processed Tweets/tweets_' +date+'.pkl', 'rb')
        tweets = pickle.load(tweet_pkl)
        tweet_pkl.close() 
        
        #initializing the sentiment and # of tweets on a date to 0 for all candidates
        listofzeros = [date] +[0] * (len(daily_candidate_sentiments_df.columns)-1)
        row_df = pd.Series(listofzeros, index=daily_candidate_sentiments_df.columns)
        daily_candidate_sentiments_df = daily_candidate_sentiments_df.append(row_df, ignore_index=True)
        daily_candidate_tweets_df = daily_candidate_tweets_df.append(row_df, ignore_index=True)
        
        i=0
        for tweet in tweets:
            i+=1
            tweets_len = str(len(tweets))
            for candidate in candidates_keywords:
                #checking if the tweet contains any keywords for that candidate
                if any(word in tweets[tweet]['tweet_text'] for word in candidates_keywords[candidate]):
                    
                    #updating the tweet count for that candidate in the daily_candidate_tweets_df dataframe
                    old_count = daily_candidate_tweets_df.loc[daily_candidate_tweets_df.Date == date, candidate].iloc[0]                   
                    daily_candidate_tweets_df.loc[daily_candidate_tweets_df.Date == date, candidate] = old_count+1
                    
                    #updating the average sentiment for that candidate in the daily_candidate_sentiments_df dataframe
                    if int(old_count)==0: daily_candidate_sentiments_df.loc[daily_candidate_sentiments_df.Date == date, candidate] = tweets[tweet]['sentiment']
                    else:
                        old_senti = daily_candidate_sentiments_df.loc[daily_candidate_sentiments_df.Date == date, candidate].iloc[0]
                        daily_candidate_sentiments_df.loc[daily_candidate_sentiments_df.Date == date, candidate] = ((float(old_senti)  * float(old_count))+ float(tweets[tweet]['sentiment']))/(float(old_count)+1)
            
            if(i%100==0): print("Processed "+str(i)+"/" +tweets_len+" tweets from " + date, end = "\r")
        
        output = open('daily_candidate_tweets_df.pkl', 'wb')
        pickle.dump(daily_candidate_tweets_df, output)
        output.close()

        output = open('daily_candidate_sentiments_df.pkl', 'wb')
        pickle.dump(daily_candidate_sentiments_df, output)
        output.close()
        print("Processed "+str(i)+"/" +tweets_len+" tweets from " + date)

                    
                    

Processed 2037990/2037990 tweets from 2020-03-09
